This notebook explores a new method of running the processing system that gets information from the FWS Threatened and Endangered Species System (TESS). It is based on a registration document in the MongoDB tesscache collection that already contains a searchURL parameter as essentially the API endpoint for the TESS service. The notebook here simply fires off and gets the return for that service as packaged through the bis.tess module's tessQuery function where all of the logic for parsing through the XML return from the service is contained.

In [1]:
from IPython.display import display
from bis2 import mlab
from bis import tess

In [7]:
bisDB = mlab.getDB("bis")
tesscache = bisDB["tesscache"]

This will need some refinement for future work. Currently, I filter to just the particular TESS documents I want to process, and I focused on GAP species for now. However, this should eventually be really simple logic to run with the most complicated part figuring out how we want to deal with versioning of the information over time. We should be able to use the built-in versioning in MongoDB with this architecture to good effect with whatever processes we decide to employ in shuttling properties to various other data stores.

In [10]:
for doc in tesscache.find({"originCollection":"gapspecies"}):
    tessData = tess.tessQuery(doc["searchURL"])
    display (tessData)
    print (tesscache.update_one({"_id":doc["_id"]},{"$set":{"tessData":tessData}},upsert=True).raw_result)